In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install mlflow
!pip install pyngrok

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import shutil

# Specify the path to the zip file
zip_file_path = '/content/drive/MyDrive/data4.zip'

# Specify the destination folder for the extracted files
extracted_folder_path = '/content/output'

# Unzip the file
shutil.unpack_archive(zip_file_path, extracted_folder_path, format='zip')



In [6]:
import os

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            total_size += os.path.getsize(file_path)
    return total_size / (1024 ** 3)  # Convert bytes to gigabytes

folder_path = "/content/output"  # Replace with the path to your folder
size_in_gb = get_folder_size(folder_path)

print(f"The size of the folder '{folder_path}' is: {size_in_gb:.2f} GB")

The size of the folder '/content/output' is: 0.31 GB


In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BookEmbeddings1").config("spark.executor.memory", "4g").config("spark.driver.memory", "4g").config("spark.executor.cores", "2").config("spark.default.parallelism", "4").getOrCreate()
df_sample = spark.read.parquet("output")


In [9]:
df_sample.count()

138724

In [14]:
pandas_df = df_sample.limit(10).toPandas()


/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider jo

In [16]:
pandas_df.to_csv('rolled_up.csv',index=False)

In [ ]:
df_sample.columns

In [4]:
import mlflow
import subprocess
from pyngrok import ngrok, conf
import getpass

In [5]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", MLFLOW_TRACKING_URI])

<Popen: returncode: None args: ['mlflow', 'ui', '--backend-store-uri', 'sqli...>

In [6]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
# mlflow will create an experiment if it doesn't exist
mlflow.set_experiment("experiment2")

<Experiment: artifact_location='/content/mlruns/2', creation_time=1707636044025, experiment_id='2', last_update_time=1707636044025, lifecycle_stage='active', name='experiment2', tags={}>

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import PCA
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F
import mlflow

# Initialize Spark session
spark = SparkSession.builder.appName("BookEmbeddings1").config("spark.executor.memory", "4g").config("spark.driver.memory", "4g").config("spark.executor.cores", "2").config("spark.default.parallelism", "4").getOrCreate()


In [51]:
# Start MLflow run
with mlflow.start_run(run_name="LinearRegression_PCA"):

    # Read the Parquet file into a DataFrame
    df_sample = spark.read.parquet("/content/output")
    df_read_parquet = df_sample#.limit(1000)

    # Assuming you have a variable named embedding_length
    embedding_length = 384  # Replace with your actual embedding length

    # Check if 'Impact' is float and convert it to double
    df_sample = df_read_parquet.withColumn('Impact', col('Impact').cast('double'))

    # Select embedding variables and 'Impact' for modeling
    #feature_columns = [f"text_embeddings{i + 1}" for i in range(embedding_length)]
    feature_columns = [item for item in df_sample.columns if item not in ['Index','publisher','Impact']]
    df_model = df_sample.select(feature_columns + ['Impact'])

    # Use PCA for dimensionality reduction to 50 components
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    pca = PCA(k=50, inputCol="features", outputCol="pca_features")

    # Train-test split
    train_df, test_df = df_model.randomSplit([0.7, 0.3], seed=42)

    # Create a Linear Regression model
    lr = LinearRegression(labelCol="Impact", featuresCol="pca_features")

    # Create a pipeline
    pipeline = Pipeline(stages=[assembler, pca, lr])

    # Define a parameter grid for tuning
    paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.0, 0.1, 0.01]).build()

    # Create a cross-validator
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=RegressionEvaluator(labelCol="Impact"),
                              numFolds=3)

    # Fit the model
    cv_model = crossval.fit(train_df)

    # Log parameters
    mlflow.log_params({
        "regParam": cv_model.bestModel.stages[2].getRegParam(),
        "elasticNetParam": cv_model.bestModel.stages[2].getElasticNetParam()
    })

    # Log metrics
    train_predictions = cv_model.transform(train_df)
    test_predictions = cv_model.transform(test_df)

    # Calculate MAPE for training set
    train_mape = train_predictions.withColumn('mape', F.abs((col('Impact') - col('prediction')) / col('Impact')) * 100)
    training_mape = train_mape.select(F.mean('mape')).collect()[0][0]
    mlflow.log_metric("training_mape", training_mape)

    # Calculate MAPE for test set
    test_mape = test_predictions.withColumn('mape', F.abs((col('Impact') - col('prediction')) / col('Impact')) * 100)
    test_mape = test_mape.select(F.mean('mape')).collect()[0][0]
    mlflow.log_metric("test_mape", test_mape)

    # Log the model
    mlflow.spark.log_model(cv_model.bestModel, "model")

    # Show MAPE on training set
    print(f"MAPE on training set: {training_mape:.2f}%")

    # Show MAPE on test set
    print(f"MAPE on test set: {test_mape:.2f}%")

MAPE on training set: 6.05%
MAPE on test set: 5.93%


In [19]:
from pyngrok import ngrok, conf
import getpass
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()
port=5000
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········


 * ngrok tunnel "https://c752-34-27-237-241.ngrok-free.app" -> "http://127.0.0.1:5000"


In [52]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F
import mlflow

# Initialize Spark session
spark = SparkSession.builder.appName("BookEmbeddings1").config("spark.executor.memory", "4g").config(
    "spark.driver.memory", "4g").config("spark.executor.cores", "2").config("spark.default.parallelism", "4").getOrCreate()

# Start MLflow run
with mlflow.start_run(run_name="LinearRegression_NoPCA"):

    # Read the Parquet file into a DataFrame
    df_sample = spark.read.parquet("/content/output")
    df_read_parquet = df_sample

    # Assuming you have a variable named embedding_length
    embedding_length = 384  # Replace with your actual embedding length

    # Check if 'Impact' is float and convert it to double
    df_sample = df_read_parquet.withColumn('Impact', col('Impact').cast('double'))

    # Select embedding variables and 'Impact' for modeling
    # feature_columns = [f"text_embeddings{i + 1}" for i in range(embedding_length)]
    feature_columns = [item for item in df_sample.columns if item not in ['Index','publisher','Impact']]
    df_model = df_sample.select(feature_columns + ['Impact'])

    # Use VectorAssembler to assemble features
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

    # Train-test split
    train_df, test_df = df_model.randomSplit([0.7, 0.3], seed=42)

    # Create a Linear Regression model
    lr = LinearRegression(labelCol="Impact", featuresCol="features")

    # Create a pipeline
    pipeline = Pipeline(stages=[assembler, lr])

    # Define a parameter grid for tuning
    paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.0, 0.1, 0.01]).build()

    # Create a cross-validator
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=RegressionEvaluator(labelCol="Impact"),
                              numFolds=3)

    # Fit the model
    cv_model = crossval.fit(train_df)

    # Log parameters
    mlflow.log_params({
        "regParam": cv_model.bestModel.stages[1].getRegParam(),
        "elasticNetParam": cv_model.bestModel.stages[1].getElasticNetParam()
    })

    # Log metrics
    train_predictions = cv_model.transform(train_df)
    test_predictions = cv_model.transform(test_df)

    # Calculate MAPE for training set
    train_mape = train_predictions.withColumn('mape', F.abs((col('Impact') - col('prediction')) / col('Impact')) * 100)
    training_mape = train_mape.select(F.mean('mape')).collect()[0][0]
    mlflow.log_metric("training_mape", training_mape)

    # Calculate MAPE for test set
    test_mape = test_predictions.withColumn('mape', F.abs((col('Impact') - col('prediction')) / col('Impact')) * 100)
    test_mape = test_mape.select(F.mean('mape')).collect()[0][0]
    mlflow.log_metric("test_mape", test_mape)

    # Log the model
    mlflow.spark.log_model(cv_model.bestModel, "model")

    # Show MAPE on training set
    print(f"MAPE on training set: {training_mape:.2f}%")

    # Show MAPE on test set
    print(f"MAPE on test set: {test_mape:.2f}%")


MAPE on training set: 5.96%
MAPE on test set: 5.86%


In [35]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F
import mlflow

# Initialize Spark session
spark = SparkSession.builder.appName("BookEmbeddings1").config("spark.executor.memory", "4g").config(
    "spark.driver.memory", "4g").config("spark.executor.cores", "2").config("spark.default.parallelism", "4").getOrCreate()

# Start MLflow run
with mlflow.start_run(run_name="DecisionTreeRegressor_NoPCA"):

    # Read the Parquet file into a DataFrame
    df_sample = spark.read.parquet("/content/output")
    df_read_parquet = df_sample

    # Assuming you have a variable named embedding_length
    embedding_length = 384  # Replace with your actual embedding length

    # Check if 'Impact' is float and convert it to double
    df_sample = df_read_parquet.withColumn('Impact', col('Impact').cast('double'))

    # Select embedding variables and 'Impact' for modeling
    feature_columns = [item for item in df_sample.columns if item not in ['Index', 'publisher', 'Impact']]
    df_model = df_sample.select(feature_columns + ['Impact'])

    # Use VectorAssembler to assemble features
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

    # Train-test split
    train_df, test_df = df_model.randomSplit([0.7, 0.3], seed=23)

    # Create a Decision Tree Regressor
    dt = DecisionTreeRegressor(labelCol="Impact", featuresCol="features")

    # Create a pipeline
    pipeline = Pipeline(stages=[assembler, dt])

    # Define a parameter grid for tuning
    paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [5, 10, 15]).build()

    # Create a cross-validator
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=RegressionEvaluator(labelCol="Impact"),
                              numFolds=3)

    # Fit the model
    cv_model = crossval.fit(train_df)

    # Log parameters
    mlflow.log_params({
        "maxDepth": cv_model.bestModel.stages[1].getMaxDepth(),
    })

    # Log metrics
    train_predictions = cv_model.transform(train_df)
    test_predictions = cv_model.transform(test_df)

    # Calculate MAPE for training set
    train_mape = train_predictions.withColumn('mape', F.abs((col('Impact') - col('prediction')) / col('Impact')) * 100)
    training_mape = train_mape.select(F.mean('mape')).collect()[0][0]
    mlflow.log_metric("training_mape", training_mape)

    # Calculate MAPE for test set
    test_mape = test_predictions.withColumn('mape', F.abs((col('Impact') - col('prediction')) / col('Impact')) * 100)
    test_mape = test_mape.select(F.mean('mape')).collect()[0][0]
    mlflow.log_metric("test_mape", test_mape)

    # Log the model
    mlflow.spark.log_model(cv_model.bestModel, "model")

    # Show MAPE on training set
    print(f"MAPE on training set: {training_mape:.2f}%")

    # Show MAPE on test set
    print(f"MAPE on test set: {test_mape:.2f}%")


MAPE on training set: 6.02%
MAPE on test set: 5.92%


In [12]:
!pip install pyspark-xgboost



ERROR: Could not find a version that satisfies the requirement pyspark-xgboost (from versions: none)
ERROR: No matching distribution found for pyspark-xgboost


In [11]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F
import mlflow
from ml.dmlc.xgboost4j.scala.spark import XGBoostRegressor

# Initialize Spark session
spark = SparkSession.builder.appName("BookEmbeddings1").config("spark.executor.memory", "4g").config(
    "spark.driver.memory", "4g").config("spark.executor.cores", "2").config("spark.default.parallelism", "4")\
    .config("spark.jars.packages", "ml.dmlc:xgboost4j-spark_2.12:1.4.0").getOrCreate()

# Start MLflow run
with mlflow.start_run(run_name="XGBoostRegressor_NoPCA"):

    # Read the Parquet file into a DataFrame
    df_sample = spark.read.parquet("/content/output")
    df_read_parquet = df_sample

    # Assuming you have a variable named embedding_length
    embedding_length = 384  # Replace with your actual embedding length

    # Check if 'Impact' is float and convert it to double
    df_sample = df_read_parquet.withColumn('Impact', col('Impact').cast('double'))

    # Select embedding variables and 'Impact' for modeling
    feature_columns = [item for item in df_sample.columns if item not in ['Index', 'publisher', 'Impact']]
    df_model = df_sample.select(feature_columns + ['Impact'])

    # Use VectorAssembler to assemble features
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

    # Train-test split
    train_df, test_df = df_model.randomSplit([0.7, 0.3], seed=23)

    # Create an XGBoost Regressor
    xgboost = XGBoostRegressor(
        featuresCol="features",
        labelCol="Impact",
        predictionCol="prediction"
    )

    # Create a pipeline
    pipeline = Pipeline(stages=[assembler, xgboost])

    # Define a parameter grid for tuning
    paramGrid = ParamGridBuilder().addGrid(xgboost.maxDepth, [5, 10, 15]).build()

    # Create a cross-validator
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=RegressionEvaluator(labelCol="Impact"),
                              numFolds=3)

    # Fit the model
    cv_model = crossval.fit(train_df)

    # Log parameters
    mlflow.log_params({
        "maxDepth": cv_model.bestModel.stages[-1].getMaxDepth(),
    })

    # Log metrics
    train_predictions = cv_model.transform(train_df)
    test_predictions = cv_model.transform(test_df)

    # Calculate MAPE for training set
    train_mape = train_predictions.withColumn('mape', F.abs((col('Impact') - col('prediction')) / col('Impact')) * 100)
    training_mape = train_mape.select(F.mean('mape')).collect()[0][0]
    mlflow.log_metric("training_mape", training_mape)

    # Calculate MAPE for the test set
    test_mape = test_predictions.withColumn('mape', F.abs((col('Impact') - col('prediction')) / col('Impact')) * 100)
    test_mape = test_mape.select(F.mean('mape')).collect()[0][0]
    mlflow.log_metric("test_mape", test_mape)

    # Log the model
    mlflow.spark.log_model(cv_model.bestModel, "model")

    # Show MAPE on the training set
    print(f"MAPE on the training set: {training_mape:.2f}%")

    # Show MAPE on the test set
    print(f"MAPE on the test set: {test_mape:.2f}%")


ModuleNotFoundError: No module named 'ml'